In [2]:
import folium
import json
from folium import plugins
import os
from folium.features import CustomIcon

from scipy.spatial import distance

from geopy.distance import geodesic
from geopy.distance import great_circle
from geopy.geocoders import Nominatim

import webbrowser

import requests
import json
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings('ignore')


In [3]:
df = pd.read_csv('final_data.csv', index_col = 0)

In [4]:
df.head()

,name,address,pro_review_score,sentiment,g_rating,appearance_scores,laty,longy,QuORP,yelp_sentiment,yelp_popularity,overall_score
0,Adrienne's Pizzabar,"54 Stone St, New York, NY 10004, USA",0.355114,0.887087,4.4,0.641275,40.704248,-74.010113,2.886126,3.172662,0.064088,19.035714
1,Alligator Lounge,"600 Metropolitan Ave, Brooklyn, NY 11211, USA",0.000000,0.876896,4.3,0.435245,40.713906,-73.948909,2.577609,2.555626,0.304762,16.657892
2,Cafe Amore's Restaurant,"147 Chambers St, New York, NY 10007, USA",0.625000,0.849441,3.3,0.435245,40.715978,-74.009694,3.216072,3.832554,0.025424,18.847135
3,Antika,"36-08 30th Ave, Astoria, NY 11103, USA",0.610795,0.012291,4.3,0.435245,40.764720,-73.917227,2.804517,3.009443,0.088090,16.040625
4,Antonio's,"318 Flatbush Ave, Brooklyn, NY 11238",1.193182,0.973098,4.4,0.782795,40.677118,-73.972672,2.065011,1.530431,0.085935,19.565671


# Create "Best Of" Map of NYC

Creates an html file folium map of the overall score

In [5]:
df_ = df.sort_values('overall_score', ascending = False).reset_index(drop = True).copy()

In [8]:
# ===================================================
#
# Creates an instance of the map centered on the
# first pizzeria and outputs an HTML file.
# The first five places have a numbered icon
#
# ===================================================

icon_image1 = 'images/pizza-1.png'
icon_image2 = 'images/pizza-2.png'
icon_image3 = 'images/pizza-3.png'
icon_image4 = 'images/pizza-4.png'
icon_image5 = 'images/pizza-5.png'
icon_image = 'images/pizza-n.png'
# shadow_image = url('leaf-shadow.png')

icon1 = CustomIcon(
    icon_image1,
    icon_size=(40, 40),
    icon_anchor=(20, 40),
    popup_anchor=(0, -40)
)

icon2 = CustomIcon(
    icon_image2,
    icon_size=(36, 36),
    icon_anchor=(18, 36),
    popup_anchor=(0, -36)
)

icon3 = CustomIcon(
    icon_image3,
    icon_size=(32, 32),
    icon_anchor=(16, 32),
    popup_anchor=(0, -32)
)

icon4 = CustomIcon(
    icon_image4,
    icon_size=(28, 28),
    icon_anchor=(14, 28),
    popup_anchor=(0, -28)
)

icon5 = CustomIcon(
    icon_image5,
    icon_size=(24, 24),
    icon_anchor=(12, 24),
    popup_anchor=(0, -24)
)

icon = CustomIcon(
    icon_image,
    icon_size=(20, 20),
    icon_anchor=(10, 20),
    popup_anchor=(0, -20)
)

nycMap = folium.Map(location=[df_.laty[0], df_.longy[0]], tiles='stamentoner', zoom_start=12)
folium.TileLayer('openstreetmap').add_to(nycMap)
folium.TileLayer('cartodbdark_matter').add_to(nycMap)
folium.LayerControl().add_to(nycMap)

for i in range(0, 5):
    html = '''<h3>''' + df_.name[i] + '''</h3><br/> <img src="file:///Users/robblatt/Python/Pizza/Final Combination/wordclouds/''' + df_.name[i][:6] + '''.png">'''
    if i == 0:
        marker = folium.Marker(
            location=[df_.laty[i],df_.longy[i]],
            icon=icon1,
            popup=html,
        ).add_to(nycMap)
    elif i == 1:
        marker = folium.Marker(
            location=[df_.laty[i],df_.longy[i]],
            icon=icon2,
            popup=html,
        ).add_to(nycMap)
    elif i == 2:
        marker = folium.Marker(
            location=[df_.laty[i],df_.longy[i]],
            icon=icon3,
            popup=html,
        ).add_to(nycMap)
    elif i == 3:
        marker = folium.Marker(
            location=[df_.laty[i],df_.longy[i]],
            icon=icon4,
            popup=html,
        ).add_to(nycMap)
    elif i == 4:
        marker = folium.Marker(
            location=[df_.laty[i],df_.longy[i]],
            icon=icon5,
            popup=html,
        ).add_to(nycMap)

for i in range(5,26):
    html = '''<h3>''' + df_.name[i] + '''</h3><br/> <img src="images/wordclouds/''' + df_.name[i][:6] + '''.png">'''
    folium.Marker(location = [df_.iloc[i]['laty'], df_.iloc[i]['longy']], 
                  popup=html, 
                  icon = CustomIcon(
                            icon_image,
                            icon_size=(20, 20),
                            icon_anchor=(10, 20),
                            popup_anchor=(0, -20)
                        )
                 ).add_to(nycMap)

for i in range(26,len(df_)):
    html = '''<h3>''' + df_.name[i] + '''</h3><br/> <img src="images/wordclouds/''' + df_.name[i][:6] + '''.png">'''
    folium.CircleMarker(
        (df_.laty[i],df_.longy[i]), 
        radius=5, weight=1, color='red', 
        fill_color='red', fill_opacity=.3,
        popup = html
    ).add_to(nycMap)

nycMap.save('overall.html')    
nycMap